In [1]:
%pip install transformers
%pip install PyPDF2

In [0]:
%rm -rf BERT_financialReportAnalysis/

In [3]:
!git clone https://github.com/francisliu2/BERT_financialReportAnalysis.git

Cloning into 'BERT_financialReportAnalysis'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 52 (delta 30), reused 36 (delta 14), pack-reused 0
Unpacking objects: 100% (52/52), done.


In [4]:
%cd BERT_financialReportAnalysis/

/content/BERT_financialReportAnalysis


In [5]:
from companytoVector_BERT import *
from corporate_profile_extrator import *

from google.colab import files
import os
import glob

os.mkdir('outputs')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Max number of tokens in one sentence: 512


In [0]:
CP = glob.glob("./data/*.pdf")

In [0]:
model = load_model()

In [0]:
success = []
failed = []
for i in range(len(CP)):
  name = CP[i].replace('.pdf', '') 
  name = name.replace('./data/', '')
  corporate_profile_text = get_corporate_profile(CP[i])

  # if corporate_profile_text == None:
  #   failed.append(name)
  # else:
  #   success.append(name)
  #   result = company2vector(corporate_profile_text, model)
  # np.save('./outputs/'+name, result)

In [0]:
a = text_tokening(corporate_profile_text, verbose=0)

In [0]:
sent = sent_tokenize(corporate_profile_text)

# Remove unwanted characters (this function should be factored out later)
for i, s in enumerate(sent):
  sent[i] = s.replace('\n', '')

sent_token = []
for s in sent:
  sent_token.append((tokenizer.encode(s, pad_to_max_length=tokenizer.max_len)))

In [0]:
tokens = []
for i in range(len(sent_token)):
  tokens.append(tokenizer.convert_ids_to_tokens(sent_token[i]))

In [52]:
sent_token

[[101,
  15169,
  3189,
  2005,
  1996,
  3361,
  2095,
  3092,
  2861,
  2233,
  2760,
  2859,
  2732,
  2833,
  2177,
  3132,
  1064,
  1015,
  1015,
  1012,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0

In [51]:
for z in list(zip(sent_token, tokens)):
  for i in range(len(z[0])):
    print(z[0][i], z[1][i])

101 [CLS]
15169 sustainability
3189 report
2005 for
1996 the
3361 financial
2095 year
3092 ended
2861 31
2233 march
2760 2018
2859 china
2732 star
2833 food
2177 group
3132 limited
1064 |
1015 1
1015 1
1012 .
102 [SEP]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PA

In [50]:
for i in range(len(z[0])):
  print(z[0][i], z[1][i])

101 [CLS]
20116 cs
2546 ##f
2290 ##g
2001 was
3205 listed
2006 on
1996 the
4937 cat
13911 ##alis
2102 ##t
2604 board
1997 of
5264 singapore
3863 exchange
3081 via
1037 a
7901 reverse
15336 takeover
1997 of
11535 brooke
4021 asia
3132 limited
2006 on
2570 22
2244 september
2325 2015
1012 .
102 [SEP]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [PAD]
0 [P

In [0]:
success

In [0]:
!zip -r ./outputs.zip ./outputs
!sleep 2
files.download('outputs.zip')

updating: outputs/ (stored 0%)
updating: outputs/China Star_Sustainability Report.npy (deflated 7%)
updating: outputs/Yongnam Annual Report 2010.npy (deflated 7%)


In [0]:
# Check time needed to run
corporate_profile_text = get_corporate_profile(CP[0])
%time result = company2vector(corporate_profile_text, model)

CPU times: user 29.9 s, sys: 2.69 s, total: 32.6 s
Wall time: 32.7 s


In [32]:
one = [1,2,3]
two = ['a','b','c']

for pairs in (list(zip(one, two))):
  print(pairs)

(1, 'a')
(2, 'b')
(3, 'c')
